For exporting to PDF:
```bash
apt-get install pandoc 
```

Install dependencies:
```bash
pip install bokeh
pip install jupyter_bokeh
```


In [31]:

import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import re
from numpy import NaN

ten_thousand_2_min = "./full-stack-test/iam_load_test_1000vus_45m_increased_timeout.csv"
df = pd.read_csv(ten_thousand_2_min, low_memory=False)
if "tracked=true" in df.extra_tags.unique():
    df = df[df["extra_tags"]=="tracked=true"]
df["timestamp"] = pd.to_datetime(df.timestamp, unit='s')

output_notebook()

df

Loading BokehJS ...

,metric_name,timestamp,metric_value,check,error,error_code,expected_response,group,method,name,proto,scenario,service,status,subproto,tls_version,url,extra_tags,metadata
0,http_reqs,2024-05-13 20:53:44,1.000000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
1,http_req_duration,2024-05-13 20:53:44,1.817000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
2,http_req_blocked,2024-05-13 20:53:44,1.820000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
3,http_req_connecting,2024-05-13 20:53:44,0.427000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
4,http_req_tls_handshaking,2024-05-13 20:53:44,0.000000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384100,data_sent,2024-05-13 21:39:16,868.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3384101,data_received,2024-05-13 21:39:16,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3384102,data_sent,2024-05-13 21:39:16,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3384103,data_received,2024-05-13 21:39:16,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:

def extract_url_prefix(url):
    if url is NaN or 'localhost' in url:
        return None
    pattern = r"(.*)(0x[a-fA-F0-9]{40})"
    match = re.match(pattern, url)
    return match.group(1) if match else url

df['normalized_url'] = df['url'].apply(extract_url_prefix)
df['normalized_url']

df = df[df['normalized_url'].notna()]

In [33]:
df.metric_name.unique()

array(['http_reqs', 'http_req_duration', 'http_req_blocked',
       'http_req_connecting', 'http_req_tls_handshaking',
       'http_req_sending', 'http_req_waiting', 'http_req_receiving',
       'http_req_failed'], dtype=object)

In [34]:
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [35]:
df[df.metric_name == "http_reqs"].groupby(by=["status", "normalized_url", "method"]).count()

metric_name  \
status normalized_url                                     method                
0.0    https://iam.staging.passport.gitcoin.co/static/... GET            2487   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST          83634   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST            779   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST          80710   
       https://iam.staging.passport.gitcoin.co/static/... GET           21046   
401.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST             39   
502.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST             18   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST             22   
       https://iam.staging.passport.gitcoin.co/static/... GET              21   
504.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST          13856   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST          32746   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST           2874   
       https://iam.staging.passport.gitcoin.co/static/... GET            9993   

                                                                  timestamp  \
status normalized_url                                     method              
0.0    https://iam.staging.passport.gitcoin.co/static/... GET          2487   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST        83634   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST          779   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        80710   
       https://iam.staging.passport.gitcoin.co/static/... GET         21046   
401.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST           39   
502.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST           18   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST           22   
       https://iam.staging.passport.gitcoin.co/static/... GET            21   
504.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST        13856   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        32746   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST         2874   
       https://iam.staging.passport.gitcoin.co/static/... GET          9993   

                                                                  metric_value  \
status normalized_url                                     method                 
0.0    https://iam.staging.passport.gitcoin.co/static/... GET             2487   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST           83634   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST             779   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST           80710   
       https://iam.staging.passport.gitcoin.co/static/... GET            21046   
401.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST              39   
502.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST              18   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST              22   
       https://iam.staging.passport.gitcoin.co/static/... GET               21   
504.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST           13856   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST           32746   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST            2874   
       https://iam.staging.passport.gitcoin.co/static/... GET             9993   

                                                                  check  \
status normalized_url                                     method          
0.0    https://iam.staging.passport.gitcoin.co/static/... GET         0   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   
       https://iam.staging.passport.gitcoin.co/api/v0.... 

In [36]:
# df['normalized_url'] = df.url.str.extract(r'(?<=http://localhost:8080)(.*)')
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [37]:
df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    # "slate gray",
    # "forest green",
    "lavender",
    "coral",
    "turquoise",
    "gold"
]


color = iter(colors)
p = figure(title="Duration (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        # p.line(group.index, group["metric_value"]["min"], legend_label=f"min - {url}", line_width=2, color=next(color))
        # p.line(group.index, group["metric_value"]["max"], legend_label=f"max - {url}", line_width=2, color=next(color))
        p.line(group.index, group["metric_value"]["mean"], legend_label=f"mean - {url}", line_width=2, color=next(color))


show(p)


In [38]:

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]


p = figure(title="Req / second (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        print(idx)
        p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])


show(p)

0
3
6
10


In [39]:

from math import pi
from bokeh.models import DatetimeTickFormatter
from collections import defaultdict

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

# dfts = dft.groupby(by=["url"]).resample("1Min").agg({
#     "metric_value": ["min", "max", "mean", "count"]
# })

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]

p = figure(title="Req / second (status != 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

p.xaxis.major_label_orientation = pi/4

# Initialize dictionaries for tracking sums per URL
url_200_sums = defaultdict(int)
url_non_200_sums = defaultdict(int)

# Loop through the grouped data and collect sums for each URL
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    # Sum up counts for each status
    total_count = group["metric_value"]["count"].sum()
    
    # Check if the status is 200 or not and add to the appropriate sum
    if int(status) == 200:
        url_200_sums[url] += total_count
    else:
        url_non_200_sums[url] += total_count

# Print summary for each individual URL
for url in url_200_sums.keys() | url_non_200_sums.keys():
    sum_200 = url_200_sums[url]
    sum_non_200 = url_non_200_sums[url]
    total = sum_200 + sum_non_200
    success_rate = (float(sum_200) / total if total > 0 else 0) * 100

    print(f"URL: {url}")
    print(f"   200       {sum_200}")
    print(f"   non-200   {sum_non_200}")
    print(f"   success {success_rate:.2f}%\n")

# Optional: If you also want to show the plots
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) != 200:
        p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])

show(p)



URL: https://iam.staging.passport.gitcoin.co/api/v0.0.0/verify
   200       80710
   non-200   2913
   success 96.52%

URL: https://iam.staging.passport.gitcoin.co/api/v0.0.0/challenge
   200       83634
   non-200   13874
   success 85.77%

URL: https://iam.staging.passport.gitcoin.co/api/v0.0.0/check
   200       779
   non-200   32768
   success 2.32%

URL: https://iam.staging.passport.gitcoin.co/static//providerBitMapInfo.json
   200       21046
   non-200   12501
   success 62.74%

